In [ ]:
import os, logging, warnings
warnings.filterwarnings("ignore", message="enable_nested_tensor is True, but self.use_nested_tensor is False")
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')
os.environ.update({
    "TF_CPP_MIN_LOG_LEVEL": "3",
    "TF_ENABLE_ONEDNN_OPTS": "0",
    "TOKENIZERS_PARALLELISM": "false",
    "CUDA_VISIBLE_DEVICES": "1"
})
logging.getLogger("tensorflow").setLevel(logging.FATAL)

os.environ["DEEPSEEK_API_KEY"] = r"sk-"
from langchain_deepseek import ChatDeepSeek
from typing import Annotated
from langchain_core.messages import BaseMessage
from langchain_core.tools import tool
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from FinTools import predict_stock_trend, evaluate_stock_risk, analyze_stock_valuation, comprehensive_stock_analysis,recommend_default_stocks
from FinRAG import fin_document_rag
from ReasearchRAG import research_document_rag
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human. Use this when your knowledge is not enough for answering."""
    human_response = interrupt({"query": query})
    return human_response["data"]

CUSTOM_TOOLS = [
    predict_stock_trend,
    evaluate_stock_risk,
    analyze_stock_valuation,
    recommend_default_stocks,
    comprehensive_stock_analysis,
    fin_document_rag,
    research_document_rag,
    human_assistance
]

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    user_name: str
    birthday: str
graph_builder = StateGraph(State)

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_base="http://localhost:8000/v1"
)
llm_with_tools=llm.bind_tools(CUSTOM_TOOLS)
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")


tool_node = ToolNode(tools=CUSTOM_TOOLS)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]},{"configurable": {"thread_id": "1"}}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break
snapshot = graph.get_state({"configurable": {"thread_id": "1"}})
print(snapshot)

In [ ]:
user_input = "please use your human_assistance tool"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()
# if interrupted
human_response = (input("please enter your reply to Agent:"))

human_command = Command(resume={"data": human_response})#using data key , because in human_assistance, we write this key

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

snapshot = graph.get_state(config)
print(snapshot)